In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from albumentations import ToTensorV2
import albumentations as A
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torchmetrics.classification import MulticlassF1Score, MulticlassAccuracy,MulticlassRecall


def seed_everything(seed: int = 42):
    random.seed(seed)          # python random
    np.random.seed(seed)       # numpy
    torch.manual_seed(seed)    # torch CPU
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

base_path=Path(r"/kaggle/input/datasets/alxmamaev/flowers-recognition/flowers")
path_list=[]

for img_path in base_path.rglob("*.jpg"):
    path_list.append({"label":img_path.parent.name,"path":img_path})

df=pd.DataFrame(path_list)
df['targets']=pd.factorize(df['label'])[0]
df=df.sample(frac=1,random_state=42).reset_index(drop=True)

train_df,tmp_df=train_test_split(df,test_size=0.3,stratify=df['label'],random_state=42)
val_df,test_df=train_test_split(tmp_df,test_size=0.4,stratify=tmp_df['label'],random_state=42)

img_aug=A.Compose([
    A.RandomResizedCrop(size=(224,224),scale=(0.8,1.0),ratio=(0.9,1.1),p=1),
    A.HorizontalFlip(p=0.3),
    A.Affine(scale=(0.9,1.1),rotate=(-15,15),border_mode=cv2.BORDER_REFLECT_101,p=0.3),
    A.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2,hue=0.03,p=0.6),
    A.CoarseDropout(num_holes_range=(1, 1), hole_height_range=(48, 48),
                    hole_width_range=(48, 48),fill=0,p=0.25)
])

tr_resnet34=A.Compose([
    A.RandomResizedCrop(size=(224,224),scale=(0.8,1.0),ratio=(0.9,1.1),p=1),
    A.HorizontalFlip(p=0.3),
    A.Affine(scale=(0.9,1.1),rotate=(-15,15),border_mode=cv2.BORDER_REFLECT_101,p=0.3),
    A.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2,hue=0.03,p=0.6),
    A.CoarseDropout(num_holes_range=(1, 1), hole_height_range=(48, 48),
                    hole_width_range=(48, 48),fill=0,p=0.25),
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_resnet34=A.Compose([
    A.Resize(224,224,p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

class FlowerCustom(Dataset):
    def __init__(self,path,targets,augment=None):
        self.path=path
        self.targets=targets
        self.augment=augment
    def __len__(self):
        return len(self.path)
    def __getitem__(self,idx):
        img=cv2.imread(self.path[idx])
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        if self.augment is None:
            raise ValueError("IMG Augment must be need")
        img=self.augment(image=img)['image']
        targets=torch.tensor(self.targets[idx],dtype=torch.long)
        return img,targets


train_custom=FlowerCustom(train_df['path'].to_list(),train_df['targets'].to_list(),
                          augment=tr_resnet34)
val_custom=FlowerCustom(val_df['path'].to_list(),val_df['targets'].to_list(),
                        augment=val_resnet34)
test_custom=FlowerCustom(test_df['path'].to_list(),test_df['targets'].to_list(),
                         augment=val_resnet34)

train_loader=DataLoader(train_custom,batch_size=32,shuffle=True,num_workers=4,pin_memory=True)
val_loader=DataLoader(val_custom,batch_size=32,shuffle=False,num_workers=4,pin_memory=True)
test_loader=DataLoader(test_custom,batch_size=32,shuffle=False,num_workers=4,pin_memory=True)

print("done")

In [ ]:
k,v=next(iter(train_loader))
k.shape

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer=nn.Sequential(
            nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),

            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),

            nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1,1)) 
        )

        self.fc_layer=nn.Sequential(
            nn.Linear(256,128),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(128),
            nn.Linear(128,5),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self,x):
        x=self.conv_layer(x)
        x=torch.flatten(x,1)
        x=self.fc_layer(x)
        return x

    def forward_logit(self,model,x): # logit값만 구하는 로
        feats=model.conv_layer(x)
        feats=torch.flatten(feats,1) # 0번 차원(B)은 그대로 두고, 1번 차원부터 끝까지 flatten하라는것. (B,1152)
        logits=model.fc_layer[:-1](feats) # 시퀀셜 인덱싱. LogSoftmax전까지만 슬라이싱
        return logits

from typing import List
from dataclasses import dataclass,field
from tqdm import tqdm

@dataclass
class History:
    training_accuracy:List[float]=field(default_factory=list)
    training_recall:List[float]=field(default_factory=list)
    training_loss:List[float]=field(default_factory=list)
    val_accuracy:List[float]=field(default_factory=list)
    val_recall:List[float]=field(default_factory=list)
    val_loss:List[float]=field(default_factory=list)
history=History()


class Trainer:
    def __init__(self,train_loader,val_loader,model,optimizer,loss_func,
                 scheduler,metric_acc,metric_rec,device,history,mode="min"):
        self.model=model
        self.train_loader=train_loader
        self.val_loader=val_loader
        self.optimizer=optimizer
        self.loss_func=loss_func
        self.scheduler=scheduler
        self.metric_acc=metric_acc
        self.metric_rec=metric_rec
        self.device=device
        self.history=history
        if mode=="max":
            self.best_value=float('-inf')
        else:
            self.best_value=float('inf')

    def training_epoch(self,epoch):
        self.metric_acc.reset()
        self.metric_rec.reset()
        self.model.train()
        loss_sum=0.0
        avg_loss=0.0
        with tqdm(total=len(self.train_loader),desc=f"training {epoch}",leave=True) as bar:
            for batch_idx,(x_train,y_train) in enumerate(self.train_loader):
                x_train=x_train.to(self.device)
                y_train=y_train.to(self.device)
                logits=self.model(x_train)
                loss=self.loss_func(logits,y_train)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                loss_sum+=loss.item()
                avg_loss=loss_sum/(batch_idx+1)
                preds=logits.argmax(dim=1)   # dim=-1과 같다. (B,12)  1번 dim 즉 행에 대해서
                self.metric_acc.update(preds, y_train)
                self.metric_rec.update(preds, y_train)
                bar.update(1)

                if batch_idx%10==0:
                    acc=self.metric_acc.compute().item()
                    recall=self.metric_rec.compute().item()
                    bar.set_postfix({"acc": acc, "recall":recall, "loss":avg_loss,"epoch":epoch})
            return self.metric_acc.compute().item(), self.metric_rec.compute().item(),avg_loss  

    def validating_epoch(self,epoch):
        self.metric_acc.reset()
        self.metric_rec.reset()
        self.model.eval()
        loss_sum=0
        avg_loss=0.0
        with tqdm(total=len(self.val_loader),desc=f"validating {epoch}", leave=True) as bar:
            with torch.no_grad():
                for batch_idx,(x_val,y_val) in enumerate(self.val_loader):
                    x_val=x_val.to(self.device)
                    y_val=y_val.to(self.device)
                    logits=self.model(x_val)
                    loss=self.loss_func(logits,y_val)

                    preds=logits.argmax(dim=-1)
                    self.metric_acc.update(preds,y_val)
                    self.metric_rec.update(preds,y_val)
                    loss_sum+=loss.item()
                    avg_loss=loss_sum/(batch_idx+1)
                    bar.update(1)
                    if batch_idx%10==0:
                        acc=self.metric_acc.compute().item()
                        recall=self.metric_rec.compute().item()
                        bar.set_postfix({"acc": acc, "recall":recall, "loss":avg_loss,"epoch":epoch})
                return self.metric_acc.compute().item(), self.metric_rec.compute().item(),avg_loss

    
    def fit(self,epochs,early_stop,path):
        stop_count=0   
        for epoch in range(epochs):
            training_accuracy,training_recall,training_loss=self.training_epoch(epoch)
            self.history.training_accuracy.append(training_accuracy)
            self.history.training_recall.append(training_recall)
            self.history.training_loss.append(training_loss)
            val_accuracy,val_recall,val_loss=self.validating_epoch(epoch)
            self.history.val_accuracy.append(val_accuracy)
            self.history.val_recall.append(val_recall)
            self.history.val_loss.append(val_loss)
            
            if self.best_value>val_loss:
                self.best_value=val_loss
                stop_count=0
                torch.save(self.model.state_dict(),os.path.join(path,f"{epoch}_{val_loss}.pt"))
            else:
                stop_count+=1
                if stop_count>=early_stop:
                    print(f"early_stopped. current epoch : {epoch}")
                    return self.history
                    
        return self.history

from torch.optim import Adam
from torchmetrics import Accuracy,Recall
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

device="cuda" if torch.cuda.is_available() else "cpu"
model=SimpleCNN()
model=model.to(device)
optimizer=Adam(model.parameters(),lr=1e-3)
loss_func = nn.NLLLoss()
acc_metric=Accuracy(task="multiclass",num_classes=5)
recall_metric=Recall(task="multiclass",num_classes=5,average="macro")
scheduler=ReduceLROnPlateau(optimizer,factor=0.1,patience=3)
model=model.to(device)
metric_rec=recall_metric.to(device)
metric_acc=acc_metric.to(device)

output_path=r"/kaggle/working/"
t=Trainer(train_loader,val_loader,model,optimizer,loss_func,scheduler,acc_metric,recall_metric,device,history,mode="min")
history=t.fit(30,5,output_path)

In [ ]:
from sklearn.metrics import confusion_matrix

best_param=torch.load(r"/kaggle/working/8_0.8227456068992615.pt")
model.load_state_dict(best_param)

class Predict:
    def __init__(self,model,test_loader,device):
        self.model=model.to(device)
        self.test_loader=test_loader
        self.device=device
        self.actual_list=[]
        self.pred_list=[]
    def predict(self):
        with tqdm(total=len(self.test_loader),desc=f"predicting",leave=True) as bar:
            self.model.eval()
            for x,y in self.test_loader:
                x=x.to(self.device)
                y=y.to(self.device)
                logits=self.model(x)
                pred=torch.argmax(logits,dim=-1)
                self.pred_list.extend(pred.detach().cpu().numpy().tolist())
                self.actual_list.extend(y.detach().cpu().numpy().tolist())
                bar.update(1)
            return self.actual_list,self.pred_list

p=Predict(model,test_loader,device)
answer,preds=p.predict()
cm=confusion_matrix(answer,preds)

In [ ]:
cm

In [ ]:
import torch
import torch.nn.functional as F

class GradCAM:
    def __init__(self, model, target_layer_idx=10 ):
        self.model = model
        self.target_layer = model.conv_layer[target_layer_idx] # CAM을 뽑을 레어어선택(주로 Conv출력선)

        self.feature_maps = None # hook으로 잡아둘 데이터공간(target layer의 출력(feature map))
        self.gradients = None   # hook으로 잡아둘 데이터공간(backward 때 그 출력에 대한 gradient)
        # None으로 설정한 이유는 아직 값이 없다는것을 명확히 하기위해. 
        # 아직 forward/backward를 안 거쳤다"는 의미의 초기 상태 표시용

        self.fwd_handle = self.target_layer.register_forward_hook(self._forward_hook) 
        # register_forward_hook은 nn.Module의 공식 API로 레이어 forward가 끝난 직후 자동 실행되는 콜백 등록 함수
        # 인자로 hook_fn을 받는다(여기선 아래의 _forward_hook => (module, input, output)을 받는 hook_fn을 받음
        
        self.bwd_handle = self.target_layer.register_full_backward_hook(self._backward_hook)
        # full_backward_hook 권장 (PyTorch 최신)-(module, input, output) 을 받는 함수를 인자로 받는다
        # 위와 통일

    def _forward_hook(self, module, inputs, output):
        # hook_fn으로 module은 타겟레이어, inputs는 (x,) <=항상 튜플로 받아야함!
        # output은 feature map : (B, C, H, W)
        # module, input, output은 PyTorch가 자동으로 넣어준다
        self.feature_maps = output

    def _backward_hook(self, module, grad_input, grad_output):
        # grad_output[0]: (B, C, H, W)
        self.gradients = grad_output[0]
        # Conv레이어는 입력이 1개, 출력이 1개. grad_output = (tensor,) 이런 구조
        # [0]으로 인덱싱해서 tuple 안에 들어있는 진짜 gradient 텐서만 꺼내는 것
        # 만약 인덱싱을 안하면 텐서가 아닌 튜플값이 들어와 연산이 안됨
        # 직관적으로 이해하긴 쉽지 않지만 메서드내에서 직접 안쓰이는데 인자들이 있는것은 pytorch가 정해준 형식이라 어쩔수없음.

    def remove_hooks(self):
        self.fwd_handle.remove()
        self.bwd_handle.remove()

    @torch.no_grad() # 계산그래프를 만들지마라는 데코레이터. 그래서 나중에 .backward()를 해도 이 연산은 추적대상이 아니다
    def _normalize(self, cam): # CAM 값을 0~1 범위로 정규화하는 코드
        # Grad-CAM은 히트맵이니까 0~1로 맞춰야 컬러맵 적용이 자연스럽게 적용
        # 다른 정규화보다 굳이 아래와 같이 하는 이유는 Grad-CAM은 “시각화용 상대 스케일”이 목적이라 min-max가 가장 적
        
        cam = cam - cam.min() # 텐서의 모든 원소에 대해 최소값을 빼는 연산. 그러면 -(-최소값)하면 최소값은 0이 된다
        cam = cam / (cam.max() + 1e-8) # 최대값으로 나눠 0~1사이로 정규화. 행여 max가 0이면 에러방지용으로 +1e-8
        return cam

    def generate(self, x, class_idx=None):
        """
        x: (1,1,28,28) tensor on device
        class_idx: None이면 예측 클래스로 CAM 생성
        return: cam (H,W) in [0,1], pred_idx, score
        """
        self.model.eval() # train모드에서는 dropout이 활성화. Grad-CAM 결정된 예측에 대해 어디를 봤는지 설명하는 것
        self.model.zero_grad(set_to_none=True)

        # forward
        out = self.model(x)  # (1,10)  현재는 log-prob
        # 학습을 다시 하는 게 아니라, 이 입력 x에 대한 forward 결과와 gradient를 얻기 위해 다시 모델에 넣는것
        # 학습은 많은 데이터에 대해 가중치를 최적화하는 과정. Grad-CAM은 특정 입력 한 장에 대한 gradient 분석
        
        pred_idx = int(out.argmax(dim=-1).item()) # 모델이 예측한 클래스번

        if class_idx is None:
            class_idx = pred_idx

        score = out[0, class_idx]   # 스칼라 (log-prob)
        # 텐서인덱싱. 첫번째 배치의 class_idx번째 클래스. 스칼라 값

        # backward: score에 대한 grad
        score.backward(retain_graph=False)
        # 특정 클래스 점수가 증가하려면, feature map의 어떤 채널이 얼마나 중요했는지 알고 싶기에
        # 이 클래스 점수에 영향을 준 feature map의 중요도(gradient)를 얻기 위한 단계

        
        # gradients: (1,C,H,W), feature_maps : (1,C,H,W)
        grads = self.gradients  # 계산 편의성과 가독성을 위해, self.에서 로컬 변수로 빼서 쓰는 것
        acts = self.feature_maps


        ### print("acts None?", acts is None, "grads None?", grads is None)
        ### print("acts min/max/mean:", acts.min().item(), acts.max().item(), acts.mean().item())
        #### print("grads min/max/mean:", grads.min().item(), grads.max().item(), grads.mean().item())


        # channel-wise weight: GAP over (H,W)
        # 핵심구현 부분 
        # grads.shape는 (1,C,H,W). dim=(2,3)은 H,W. 각 채널 안에서 공간 전체 평균을 내는 것
        # (1,C)로 변환. 채널 하나당 평균값 1개로 요약됨
        # torch의 mean은 기본적으로 평균낸 차원을 없앤다. 
        # 나중에 곱셈을 해야 하니까 차원을 그대로 유지하기 위해 keepdim=True
        weights = grads.mean(dim=(2,3), keepdim=True)  # (1,C,1,1)


        # weights.shape = (1, C, 1, 1)
        # acts.shape(feature_map) = (1, C, H, W)
        # weights * acts => broadcating으로 (1,C,H,W)
        # 각 채널의 전체 feature map에 그 채널 weight를 곱하는 것
        # .sum => 해당 차원(dim=1, 즉 채널차원)을 모두 더해서 하나의 지도 생성
        # keepdim=True로 채널차원 1로 유지. torch의 sum도 해당차원이 사라지나 keepdim해서 그값을 거기에 크기 1짜리 차원으로 유지
        cam = (weights * acts).sum(dim=1, keepdim=True)  # (1,1,H,W)

        # ReLU - 음수제거하기 위해. relu를 안하면 heatmap에 음수 영역도 포함됨
        # 시각적으로 덜 직관적. Grad-CAM은 설명을 단순화하기 위해 positive evidence만 남기는 것
      ###   print("pre-ReLU cam min/max/mean:",
      ### cam.min().item(), cam.max().item(), cam.mean().item())
      ###   cam = F.relu(cam)

        # input size로 업샘플
        # Grad-CAM을 이미지 위에 덮어씌우려면 입력이미지와 같은 크기여야 한다
        # target layer의 feature map 크기는 Conv와 Pool을 거치면 7x7,14x14 등등이 될수있다
        # mode="bilinear" 업샘플링 방식지정. 부드럽게 보간
        # align_corners=False. bilinear 보간 시 좌표 정렬 방식 옵션.False → 일반적으로 더 안정적
        cam = F.interpolate(cam, size=(x.shape[2], x.shape[3]), mode="bilinear", align_corners=False)

        # interpolate 이후 cam.shape = (1, 1, H, W)
        # cam[0,0] => 첫 번째 batch의 첫번째 채널선택 => (H,W)
        # 배치가 1이고 채널도 1이므로, cam[0,0]은 단순히 차원 정리용 선택
        cam = cam[0,0].detach()
        cam = self._normalize(cam)

        return cam.cpu(), pred_idx, float(score.detach().cpu().item())

In [ ]:
images, labels = next(iter(test_loader))      # images: (B,1,28,28)
x = images[0:1].to(device)                    # (1,1,28,28)
y = labels[0].item()

gradcam = GradCAM(model, target_layer_idx=8)

cam, pred_idx, score = gradcam.generate(x)  # cam: (H,W) on cpu
gradcam.remove_hooks()
print("done")

In [ ]:
import matplotlib.pyplot as plt

# 원본 이미지 (cpu로)
img = x[0, 0].detach().cpu()   # (28,28)

plt.figure(figsize=(10,3))

# 1) 원본
plt.subplot(1,3,1)
plt.title(f"Input (label={y})")
plt.imshow(img, cmap="gray")
plt.axis("off")

# 2) CAM 히트맵
plt.subplot(1,3,2)
plt.title(f"Grad-CAM (pred={pred_idx})")
plt.imshow(cam, cmap="jet")
plt.axis("off")

# 3) 오버레이
plt.subplot(1,3,3)
plt.title(f"Overlay (score={score:.3f})")
plt.imshow(img, cmap="gray")
plt.imshow(cam, cmap="jet", alpha=0.45)  # alpha로 투명도 조절
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch

def to_numpy_img(x: torch.Tensor):
    """
    x: (1,1,H,W) or (1,H,W) or (H,W)
    return: (H,W) float in [0,1]
    """
    if x.ndim == 4:
        x = x[0, 0]
    elif x.ndim == 3:
        x = x[0]
    x = x.detach().float().cpu()
    x = x - x.min()
    x = x / (x.max() + 1e-8)
    return x.numpy()

def overlay_cam_on_grayscale(img_hw01: np.ndarray, cam_hw01: np.ndarray, alpha=0.45, cmap_name="jet"):
    """
    img_hw01: (H,W) in [0,1]
    cam_hw01: (H,W) in [0,1]
    return: (H,W,3) in [0,1] (overlayed RGB)
    """
    # CAM -> 컬러(RGB)
    heat = cm.get_cmap(cmap_name)(cam_hw01)[..., :3]  # (H,W,3)

    # grayscale -> RGB
    img_rgb = np.repeat(img_hw01[..., None], 3, axis=2)  # (H,W,3)

    # overlay
    out = (1 - alpha) * img_rgb + alpha * heat
    out = np.clip(out, 0, 1)
    return out

@torch.no_grad()
def get_pred_conf(out: torch.Tensor, pred_idx: int):
    """
    out: (1,num_classes) logits 또는 log-prob
    return: softmax 확률(0~1)
    """
    prob = torch.softmax(out, dim=-1)[0, pred_idx].item()
    return prob

def show_gradcam_grid(
    model,
    gradcam: "GradCAM",
    images,               # iterable of torch.Tensor each: (1,1,H,W) or (1,H,W)
    labels=None,          # optional iterable of int
    device=None,
    rows=4,
    cols=4,
    alpha=0.45,
    cmap_name="jet",
    class_idx=None,       # None이면 예측 클래스로 CAM
    title_prefix="",
):
    model.eval()
    n = rows * cols

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3.2, rows * 3.2))
    axes = np.array(axes).reshape(-1)

    for i in range(n):
        ax = axes[i]
        ax.axis("off")

        if i >= len(images):
            continue

        x = images[i]
        if x.ndim == 3:  # (1,H,W) -> (1,1,H,W)
            x = x.unsqueeze(1)
        if device is not None:
            x = x.to(device)

        # CAM 생성
        cam, pred_idx, score = gradcam.generate(x, class_idx=class_idx)

        # 오버레이 생성
        img_hw01 = to_numpy_img(x)           # (H,W)
        cam_hw01 = cam.numpy()               # (H,W) already 0~1
        overlay = overlay_cam_on_grayscale(img_hw01, cam_hw01, alpha=alpha, cmap_name=cmap_name)

        ax.imshow(overlay)

        # (옵션) 예측/정답 표시
        gt = None if labels is None else int(labels[i])
        # score는 logit/log-prob일 수 있으니 softmax 확률도 같이 보고 싶으면 아래처럼:
        with torch.no_grad():
            out = model(x)
            conf = get_pred_conf(out, pred_idx)

        if gt is None:
            ax.set_title(f"{title_prefix}pred={pred_idx} (p={conf:.2f})", fontsize=10)
        else:
            ax.set_title(f"{title_prefix}gt={gt} / pred={pred_idx} (p={conf:.2f})", fontsize=10)

    plt.tight_layout()
    plt.show()

In [ ]:
device = next(model.parameters()).device  # 모델 device
gc = GradCAM(model, target_layer_idx=10)

# 예: test_loader에서 16장만 확보
images = []
labels = []
for x, y in test_loader:
    # x: (B,1,H,W)
    for j in range(x.size(0)):
        images.append(x[j:j+1])  # (1,1,H,W)
        labels.append(int(y[j].item()))
        if len(images) >= 16:
            break
    if len(images) >= 16:
        break

show_gradcam_grid(model, gc, images, labels=labels, device=device, rows=4, cols=4, alpha=0.45)
gc.remove_hooks()

### 틀린이미지만 Grad-CAM

In [ ]:
device = next(model.parameters()).device
gc = GradCAM(model, target_layer_idx=10)

wrong_imgs, wrong_lbls = [], []
model.eval()

for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = out.argmax(dim=1)
    wrong_mask = (pred != y)

    if wrong_mask.any():
        idxs = wrong_mask.nonzero(as_tuple=False).squeeze(1).tolist()
        for j in idxs:
            wrong_imgs.append(x[j:j+1].detach().cpu())  # 나중에 show에서 device로 올림
            wrong_lbls.append(int(y[j].item()))
            if len(wrong_imgs) >= 16:
                break
    if len(wrong_imgs) >= 16:
        break

show_gradcam_grid(model, gc, wrong_imgs, labels=wrong_lbls, device=device, rows=4, cols=4, alpha=0.45)
gc.remove_hooks()